<a href="https://colab.research.google.com/github/davisdw/Lending_Tree_Loan_Prediction_Analysis/blob/main/pyspark_data_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'

import os

spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
%pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,075 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,522 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restr

In [2]:
# Mounted Google Drive to access Data Source *DO NOT RERUN*
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pyspark
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Import accepted dataset, cache temp table and partitioned/export the data into CSV

In [4]:
from pyspark import SparkFiles

url = "/content/drive/MyDrive/Lending_Club_Data_Source/accepted_2007_to_2018Q4.csv"

spark.sparkContext.addFile(url)
loan_accept_df = spark.read.csv(SparkFiles.get("accepted_2007_to_2018Q4.csv"), sep=",", header=True)

# Display the Data
loan_accept_df.show()

+--------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+------------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-

In [5]:
# Create Temp View Tables
loan_accept_df.createOrReplaceTempView("accepted")

In [6]:
# Cache the accepted dataset table
spark.sql("cache table accepted")

DataFrame[]

In [7]:
# Truncate un-needed columns from the "accepted" table
# Tables that we're keeping -->>:loan_amnt, int_rate, emp_title, emp_length,
# home_ownership, annual_inc, issue_d, loan_status, purpose, addr_state, dti,fico_range_low, fico_range_high

a = """ SELECT
        loan_amnt,
        int_rate,
        emp_title,
        emp_length,
        home_ownership,
        annual_inc,
        issue_d,
        loan_status,
        purpose,
        addr_state,
        dti,fico_range_low,
        fico_range_high
        FROM accepted
    """

spark.sql(a).show()


+---------+--------+--------------------+----------+--------------+----------+--------+-----------+------------------+----------+-----+--------------+---------------+
|loan_amnt|int_rate|           emp_title|emp_length|home_ownership|annual_inc| issue_d|loan_status|           purpose|addr_state|  dti|fico_range_low|fico_range_high|
+---------+--------+--------------------+----------+--------------+----------+--------+-----------+------------------+----------+-----+--------------+---------------+
|   3600.0|   13.99|             leadman| 10+ years|      MORTGAGE|   55000.0|Dec-2015| Fully Paid|debt_consolidation|        PA| 5.91|         675.0|          679.0|
|  24700.0|   11.99|            Engineer| 10+ years|      MORTGAGE|   65000.0|Dec-2015| Fully Paid|    small_business|        SD|16.06|         715.0|          719.0|
|  20000.0|   10.78|        truck driver| 10+ years|      MORTGAGE|   63000.0|Dec-2015| Fully Paid|  home_improvement|        IL|10.78|         695.0|          699.0

In [8]:
# Partitioning into saller datasets and output the data into multiple folders
loan_accept_df.write.partitionBy("issue_d").mode("overwrite").csv("accepted_parquet_csv")


In [10]:
#Test by reading the partioned data
readPartitionData = spark.read.option("header", True)\
.csv("/content/accepted_parquet_csv/issue_d=Apr-2008")
readPartitionData.printSchema()
readPartitionData.show()

root
 |-- 345705: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- 7500.02: string (nullable = true)
 |-- 7500.03: string (nullable = true)
 |-- 214.02: string (nullable = true)
 |-- 36 months: string (nullable = true)
 |-- 8.0: string (nullable = true)
 |-- 235.03: string (nullable = true)
 |-- A: string (nullable = true)
 |-- A3: string (nullable = true)
 |-- JEK: string (nullable = true)
 |-- 7 years: string (nullable = true)
 |-- MORTGAGE: string (nullable = true)
 |-- 45000.0: string (nullable = true)
 |-- Not Verified: string (nullable = true)
 |-- Fully Paid: string (nullable = true)
 |-- n16: string (nullable = true)
 |-- https://lendingclub.com/browse/loanDetail.action?loan_id=345705: string (nullable = true)
 |-- Loan is for working capital for starting up a pawn shop.   20% of the United states population does not have a bank.  Lending and credit services are provided to individuals who do not have cash resources or access to credit to meet short term cash ne

In [11]:
# Read the entire csv partition collectively
partition_accepted_df = spark.read.format("csv").option("header", True).load("/content/accepted_parquet_csv")

In [12]:
# Display the partitions csv
partition_accepted_df.show()

+--------+----+-------+-------+-------+---------+-----+-------+---+---+--------------------+---------+--------+--------+---------------+------------------+---+-----------------------------------------------------------------+----+------------------+--------------------+-----+---+-----+-----+--------+-----+-----+-----+------+----+----+-----+--------+----+----+---+--------+--------+------------------+--------+--------+-------+-----+-------+---------+--------+--------+--------+--------+-----+-----+-----+------+-----+----------+----+----+----+-----+-----+--------+-----+-----+-----+-----+------+--------+----+-----+-----+-------+----+--------+-----+-----+-----+-----+-------+-------+----+-----+-----+-----+-----+------+------+-----+-----+------+------+------+-----+-----+-----+-----+----+----+-----+----+------+----+------+------+------+------+-----+-----+------+------+--------+--------+-------+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----------

In [13]:
#Un-cached the accepted table
spark.sql("uncache table accepted")

DataFrame[]

In [14]:
# Check to see if the table is cached
if spark.catalog.isCached('accepted'):
  print('accepted dataset table is cached')
else:
  print('accepted dataset table is not there')

accepted dataset table is not there


Import rejected dataset, cache temp table and partitioned/export the data into CSV

In [15]:
# Read in the loan rejection data csv

url_2 = "/content/drive/MyDrive/Lending_Club_Data_Source/rejected_2007_to_2018Q4.csv"

spark.sparkContext.addFile(url_2)
loan_reject_df = spark.read.csv(SparkFiles.get("rejected_2007_to_2018Q4.csv"), sep=',', header=True)

loan_reject_df.show()

+-------------+----------+--------------------+----------+------+--------+-----+-----------------+-----------+
|amt_requested|      date|             purpose|risk_score|   dti|zip_code|state|employment_length|policy_code|
+-------------+----------+--------------------+----------+------+--------+-----+-----------------+-----------+
|       1000.0|2007-05-26|Wedding Covered b...|     693.0|   10%|   481xx|   NM|          4 years|        0.0|
|       1000.0|2007-05-26|  Consolidating Debt|     703.0|   10%|   010xx|   MA|         < 1 year|        0.0|
|      11000.0|2007-05-27|Want to consolida...|     715.0|   10%|   212xx|   MD|           1 year|        0.0|
|       6000.0|2007-05-27|             waksman|     698.0|38.64%|   017xx|   MA|         < 1 year|        0.0|
|       1500.0|2007-05-27|              mdrigo|     509.0| 9.43%|   209xx|   MD|         < 1 year|        0.0|
|      15000.0|2007-05-27|          Trinfiniti|     645.0|    0%|   105xx|   NY|          3 years|        0.0|
|

In [16]:
loan_reject_df.createOrReplaceTempView("rejected")

In [18]:
# Cache the Rejected dataset table
spark.sql("cache table rejected")

DataFrame[]

In [19]:
# Truncate un-needed columns from the "rejected" table
# Tables that we're keeping -->>: amt_requested|date|purpose|risk_score|dti|zip_code|state|employment_length|policy_code

b = """ SELECT
        amt_requested,
        date,
        purpose,
        risk_score,
        dti,
        zip_code,
        state,
        employment_length,
        policy_code
        FROM rejected
    """

spark.sql(b).show()

+-------------+----------+--------------------+----------+------+--------+-----+-----------------+-----------+
|amt_requested|      date|             purpose|risk_score|   dti|zip_code|state|employment_length|policy_code|
+-------------+----------+--------------------+----------+------+--------+-----+-----------------+-----------+
|       1000.0|2007-05-26|Wedding Covered b...|     693.0|   10%|   481xx|   NM|          4 years|        0.0|
|       1000.0|2007-05-26|  Consolidating Debt|     703.0|   10%|   010xx|   MA|         < 1 year|        0.0|
|      11000.0|2007-05-27|Want to consolida...|     715.0|   10%|   212xx|   MD|           1 year|        0.0|
|       6000.0|2007-05-27|             waksman|     698.0|38.64%|   017xx|   MA|         < 1 year|        0.0|
|       1500.0|2007-05-27|              mdrigo|     509.0| 9.43%|   209xx|   MD|         < 1 year|        0.0|
|      15000.0|2007-05-27|          Trinfiniti|     645.0|    0%|   105xx|   NY|          3 years|        0.0|
|

In [21]:
loan_reject_df.write.partitionBy("date").mode("overwrite").csv("rejected_parquet_csv")

In [22]:
#Test by reading the partioned data
readPartitionData = spark.read.option("header", True)\
.csv("/content/rejected_parquet_csv/date=2007-05-26")
readPartitionData.printSchema()
readPartitionData.show()

root
 |-- 1000.0: string (nullable = true)
 |-- Wedding Covered but No Honeymoon: string (nullable = true)
 |-- 693.0: string (nullable = true)
 |-- 10%: string (nullable = true)
 |-- 481xx: string (nullable = true)
 |-- NM: string (nullable = true)
 |-- 4 years: string (nullable = true)
 |-- 0.0: string (nullable = true)

+------+--------------------------------+-----+---+-----+---+--------+---+
|1000.0|Wedding Covered but No Honeymoon|693.0|10%|481xx| NM| 4 years|0.0|
+------+--------------------------------+-----+---+-----+---+--------+---+
|1000.0|              Consolidating Debt|703.0|10%|010xx| MA|< 1 year|0.0|
+------+--------------------------------+-----+---+-----+---+--------+---+



In [23]:
# Read the entire csv partition collectively
partition_rejected_df = spark.read.format("csv").option("header", True).load("/content/rejected_parquet_csv")

In [24]:
# display the full partitioned data table
partition_rejected_df.show()

+-------+--------------------+-----+------+-----+---+---------+---+----------+
|11975.0|       Car financing|  _c2|17.88%|191xx| PA| < 1 year|0.0|      date|
+-------+--------------------+-----+------+-----+---+---------+---+----------+
| 5500.0|  Debt consolidation| NULL|11.88%|973xx| OR| < 1 year|0.0|2018-12-04|
|40000.0|         Home buying|591.0|99.59%|770xx| TX| < 1 year|0.0|2018-12-04|
|30000.0|      Major purchase| NULL|  100%|544xx| WI|     NULL|0.0|2018-12-04|
| 3000.0|  Debt consolidation| NULL|33.11%|551xx| MN| < 1 year|0.0|2018-12-04|
|20000.0|  Debt consolidation| NULL|26.57%|750xx| TX| < 1 year|0.0|2018-12-04|
| 6000.0|  Debt consolidation| NULL|30.82%|482xx| MI| < 1 year|0.0|2018-12-04|
| 1000.0|  Debt consolidation| NULL| 5.67%|452xx| OH| < 1 year|0.0|2018-12-04|
| 5000.0|               Other| NULL| 7.66%|080xx| NJ| < 1 year|0.0|2018-12-04|
|40000.0|  Debt consolidation| NULL|25.97%|606xx| IL| < 1 year|0.0|2018-12-04|
| 1000.0|  Debt consolidation| NULL| 1.33%|906xx| CA

In [25]:
# Uncache the rejected dataset table
spark.sql("uncache table rejected")

DataFrame[]

In [26]:
# Check to see if the table is cached
if spark.catalog.isCached('rejected'):
  print('rejected dataset table is cached')
else:
  print('rejected dataset table is not there')

rejected dataset table is not there
